# app xml 파일 쪼개기

### 각 application이 하나의 xml 파일 그 자체    
__=> 하나씩 잘라야 함__

In [1]:
#-*- coding: utf-8 -*-
import os
import re
import csv
import sys
import xml.etree.ElementTree as et
import datetime as dt
import numpy as np
import math

reload(sys)
sys.setdefaultencoding('utf-8')  # to process name of assignee in unicode 
os.chdir('J:/data')

In [ ]:
f=open('sample.xml', 'r')
endtag_regex = re.compile('^<!DOCTYPE (.*) SYSTEM')
endtag = ''
doc=''

In [ ]:
line = f.readline()
doc += line

In [ ]:
endtag

In [ ]:
endtag_regex.findall(line)

In [ ]:
endtag = endtag_regex.findall(line) if not endtag else endtag
endtag

In [ ]:
not endtag

In [ ]:
doc

In [ ]:
terminate = re.compile('^</{0}>'.format(endtag[0]))

In [ ]:
terminate.findall(line)

if not ~ else? 이게 뭐지?

In [ ]:
if not endtag == True:
    endtag = endtag_regex.findall(line)
else:
    endtag

endtag

아하 이렇게 쓰기 싫어서 그냥 한 줄로 끝냈구만!  
함수 이해 완료

In [2]:
def extract_xml_strings(filename):
    """
    Given a string [filename], opens the file and returns a generator
    that yields tuples. A tuple is of format (year, xmldoc string). A tuple
    is returned for every valid XML doc in [filename]
    """
    # search for terminating XML tag
    endtag_regex = re.compile('^<!DOCTYPE (.*) SYSTEM')
    endtag = ''
    with open(filename, 'r') as f:
        doc = ''  # initialize current XML doc to empty string
        for line in f:
            doc += line
            endtag = endtag_regex.findall(line) if not endtag else endtag   #is there any endtags?
            if not endtag:
                continue      #if there's no end tag, keep the loop going
            terminate = re.compile('^</{0}>'.format(endtag[0]))    #if there's a end tag, then check whether the doc is finished or not
            if terminate.findall(line):     #if there's a terminator
                yield (doc)                 #export doc and reinitialize a doc
                endtag = ''
                doc = ''

In [ ]:
sample = extract_xml_strings('sample.xml')

In [ ]:
for x in sample:
    print x

good~ 이젠 sample.next 해서 각 doc을 parsing, App Np, DN, assignee, class 추출하자

# xml 자동으로 불러오기

application xml 파일이 1주일 단위로 구성  
ex) ipab20070308, ipab20070315, ...  
규칙에 맞게 자동으로 불러오도록 만들자

먼저 날짜 다루기

In [ ]:
date = '20070301'
date = dt.datetime.strptime(date,'%Y%m%d')
date

In [ ]:
date += dt.timedelta(weeks=1)
date

In [ ]:
date=date.strftime('%Y%m%d')
date

filename 만들기

In [ ]:
filename = 'ipab'+ date + '.xml'
filename

In [ ]:
sample = extract_xml_strings(filename)

# 각 XML에서 원하는 element 뽑기

App NO, DN, Assignee, Class, Filed date, Country 필요

일단 각각 element가 뭔지 보면 (등장 순서대로)  
DN : us-bibliographic-data-application / publication-reference / document-id / doc-number  
APP No : us-bibliographic-data-application / application-reference / documnet-id / doc-number  
Filed date : us-bibliographic-data-application / application-reference / documnet-id / date  
Class : us-bibliographic-data-application / classification-national / main-classification  
__** issue : 3digit에서 0 제거 안 함__  ex) 312/29.1 = 312029001  
Assignee : us-bibliographic-data-application / assignees / addressbook / orgname  
Country : us-bibliographic-data-application / assignees / addressbook / address / country  

In [ ]:
doc1 = sample.next()
doc2 = sample.next()

In [ ]:
root = et.fromstring(doc1)
root

list 만들어서 각 정보들 넣은 후, csv로 바꾸자  
dn, app no, assignee, country, date 담은 csv  
dn, app no, class 담은 csv  

In [ ]:
for n in root.iter('doc-number'):
    dn = n.text

In [ ]:
n = root.iter('doc-number')
dn = n.next().text
appno = (lambda x: x[0:2] + '/' + x[2:8])(n.next().text)
print dn, type(dn), appno, type(appno)

In [ ]:
# DN, APP NO, country, date 얻기
dn = root[0][0][0][1].text
appno = (lambda x: x[0:2] + '/' + x[2:8])(root[0][1][0][1].text)
date = (lambda x: x[0:4] + '-' + x[4:6] + '-' + x[6:8])(root[0][1][0][2].text)

aline = [dn,appno,date]
cline = [dn,appno]

In [ ]:
# Assignee 얻기 (assignee는 복수일 수 있으니 iter 사용)
for ass in root.iter('assignee'):
    for name in ass.iter("orgname"):
        aline.append(name.text)
    for country in ass.iter("country"):
        aline.append(country.text)

aline.append('private') if len(aline)==3 else aline
aline

In [3]:
def digitize(cls):
    try :
        return str(int(cls))
    except ValueError:
        if cls[0] == '0':
            cls=cls.replace('0','')
        return cls

In [4]:
def classmaker(cls,line):
    cls=cls.replace(' ','0')
    if len(cls) < 9:
        for i in range(9-len(cls)):
            cls += '0'

    line.append(digitize(cls[0:3]))
    if digitize(cls[6:9])=='0':
        line.append(digitize(cls[3:6]))
    else:
        sub = (lambda x,y: x + '.' + y)(digitize(cls[3:6]),digitize(cls[6:9]))
        line.append(sub)

In [ ]:
# Class 얻기 (class는 복수일 수 있으니 iter 사용)
for ass in root.iter('main-classification'):
    rawclass = ass.text
    cline.append(classmaker(rawclass[0:3]))
    if not int(rawclass[6:9]):
        cline.append(classmaker(rawclass[3:6]))
    else:
        sub = (lambda x,y: x + '.' + y)(classmaker(rawclass[3:6]),classmaker(rawclass[6:9]))
        cline.append(sub)

cline

02 ~ 04랑 05 ~ 13이 구조적으로 완전 다르네

In [5]:
class AllEntities:
    def __getitem__(self, key):
        #key is your entity, you can do whatever you want with it here
        return key

In [ ]:
parser = et.XMLParser()
parser._parser.UseForeignDTD(True)
parser.entity = AllEntities()
root = et.fromstring(doc1,parser=parser)
root

In [6]:
date = '20070308'  #initializing date

with open('app_profile.csv', 'wb') as prf, open('app_class.csv', 'wb') as cls:
    prf_writer = csv.writer(prf,quoting=csv.QUOTE_ALL)
    cls_writer = csv.writer(cls,quoting=csv.QUOTE_ALL)
    
    prf_writer.writerow(['number','id','date','assignee','country'])
    cls_writer.writerow(['number','id','mainclass','subclass','further_m','further_s'])
    
    while not date=='20070315':
        filename = 'xmls/ipab'+ date + '.xml'   #making filename
        xml = extract_xml_strings(filename)   #load and split whole xml into app_xmls  
        for doc in xml:
            parser = et.XMLParser()
            parser.parser.UseForeignDTD(True)
            parser.entity = AllEntities()
            root = et.fromstring(doc, parser=parser)
            n = root.iter('doc-number')
            dn = n.next().text
            appno = (lambda x: x[0:2] + '/' + x[2:8])(n.next().text)
            
            n = root.iter('filing-date')
            fdate = (lambda x: x[0:4] + '-' + x[4:6] + '-' + x[6:8])(n.next().text)
            
            aline = [dn,appno,fdate]
            cline = [dn,appno]
            
            for ass in root.iter('assignee'):   #using iter because of multi-assignee apps 
                name = ass.iter("organization-name").next().text
                aline.append(name)
                
                for n in ass.iter("country-code"):    #to deal with country-missed assignees
                    country = n.text
                    aline.append(country)    
                
            aline.append('private') if len(aline)==3 else aline
            
            for cls in root.iter('uspc'):  #using iter because of multi-class apps
                rawclass = cls[0].text + cls[1].text
                classmaker(rawclass,cline)           
                          
            prf_writer.writerow(aline)
            cls_writer.writerow(cline)
        
        date = dt.datetime.strptime(date,'%Y%m%d')   #Let's move on to next xml
        date += dt.timedelta(weeks=1)
        date = date.strftime('%Y%m%d')            

In [7]:
date = '20070315'  #initializing date

with open('app_profile.csv', 'ab') as prf, open('app_class.csv', 'ab') as cls:
    prf_writer = csv.writer(prf,quoting=csv.QUOTE_ALL)
    cls_writer = csv.writer(cls,quoting=csv.QUOTE_ALL)
    
    while not date=='20070329':
        filename = 'xmls/ipab'+ date + '.xml'   #making filename
        xml = extract_xml_strings(filename)   #load and split whole xml into app_xmls  
        for doc in xml:
            root = et.fromstring(doc)
            n = root.iter('doc-number')
            dn = n.next().text
            appno = (lambda x: x[0:2] + '/' + x[2:8])(n.next().text)
            fdate = (lambda x: x[0:4] + '-' + x[4:6] + '-' + x[6:8])(root[0][1][0][2].text)
                
            aline = [dn,appno,fdate]
            cline = [dn,appno]
            
            for ass in root.iter('assignee'):   #using iter because of multi-assignee apps 
                name = ass.iter("orgname").next().text
                aline.append(name)
                
                for n in ass.iter("country"):    #to deal with country-missed assignees
                    country = n.text
                    aline.append(country)    
                
            aline.append('private') if len(aline)==3 else aline
            
            for cls in root.iter('main-classification'):  #using iter because of multi-class apps
                rawclass = cls.text
                classmaker(rawclass,cline)
            
            for cls in root.iter('further-classification'):  #using iter because of multi-class apps
                rawclass = cls.text
                classmaker(rawclass,cline)
                          
            prf_writer.writerow(aline)
            cls_writer.writerow(cline)
        
        date = dt.datetime.strptime(date,'%Y%m%d')   #Let's move on to next xml
        date += dt.timedelta(weeks=1)
        date = date.strftime('%Y%m%d')     

# numpy로 grant dummy 처리하기

merge 후 granted_dum_i, granted_dum_i 중 __하나만 NaN 일 때__ 정상  
둘 다 NaN일 경우 (matching 실패) or 둘 다 NaN 아닐 경우 (matching 중복) 처리 필요  
=> numpy array로 처리 후 다시 갖다 붙이자

In [2]:
# dummy 뒷처리

dum = np.genfromtxt('grtdum_temp.csv',delimiter=',',dtype=float,names=True)
dum
    

array([(  1.,  nan), (  0.,  nan), ( nan,  nan), ( nan,  nan),
       ( nan,   0.), ( nan,   1.), ( nan,   2.), (  1.,   1.),
       (  0.,   0.), (  1.,   2.), (  0.,   2.), (  1.,   2.),
       (  0.,   2.), (  1.,   1.), (  1.,   0.), (  0.,   1.),
       (  0.,   1.), (  1.,  nan), (  0.,  nan)], 
      dtype=[('granted_dum_i', '<f8'), ('granted_dum_n', '<f8')])

In [5]:
arr=dum[0]
arr[0]

1.0

In [18]:
def granted_dum(arr):
    a = arr[0] + arr[1]
    if math.isnan(a):
        if not math.isnan(arr[0]):
            a = arr[0]
        else:
            a = arr[1] if not math.isnan(arr[1]) else a
            a = np.nan if a==2 else a
     
    else:
        if a==2:
            if arr[0]==arr[1]:
                a=1
            
            else:
                a=0
        
        elif a==3:
            a=1
            
    return a

In [19]:
vfunc = np.vectorize(granted_dum)
result = vfunc(dum)
np.savetxt('result.csv', result,delimiter=',', fmt='%1.1f', header='granted_dum', comments='')

In [20]:
result

array([  1.,   0.,  nan,  nan,   0.,   1.,  nan,   1.,   0.,   1.,   0.,
         1.,   0.,   1.,   1.,   1.,   1.,   1.,   0.])

In [23]:
np.savetxt('result.csv', result,delimiter=',', fmt='%1.1f', header='granted_dum', comments='')

# df.iterrow 사용해보기

In [1]:
import os
import csv
import pandas as pd

os.chdir('J:/data')

df = pd.read_csv('ass.csv')

In [2]:
with open('pid_private.csv','rb') as prv:
    reader = csv.reader(prv)
    reader.next()
    line = reader.next()
        
    for i, row in df.iterrows():
        a=line[0]
        if row['patent_id'] == int(line[0]):
            df.loc[i,'assignee'] = 'private'
            line = reader.next()           

StopIteration: 

In [3]:
df

,patent_id,assignee
0,1,private
1,2,sanvk
2,3,vrji
3,4,vkf
4,5,regh
5,6,private
6,7,gnf
7,8,private
8,9,private
9,10,fnj


# Patent xml에서 appl citaiton 불러오기

In [1]:
#-*- coding: utf-8 -*-
import os
import re
import csv
import sys
import xml.etree.ElementTree as et
import datetime as dt

reload(sys)
sys.setdefaultencoding('utf-8')  # to process name of assignee in unicode 
os.chdir('J:/data')

In [2]:
def extract_xml_strings(filename):
    """
    Given a string [filename], opens the file and returns a generator
    that yields tuples. A tuple is of format (year, xmldoc string). A tuple
    is returned for every valid XML doc in [filename]
    """
    # search for terminating XML tag
    endtag_regex = re.compile('^<!DOCTYPE (.*) SYSTEM')
    endtag = ''
    with open(filename, 'r') as f:
        doc = ''  # initialize current XML doc to empty string
        for line in f:
            doc += line
            endtag = endtag_regex.findall(line) if not endtag else endtag   #is there any endtags?
            if not endtag:
                continue      #if there's no end tag, keep the loop going
            terminate = re.compile('^</{0}>'.format(endtag[0]))    #if there's a end tag, then check whether the doc is finished or not
            if terminate.findall(line):     #if there's a terminator
                yield (doc)                 #export doc and reinitialize a doc
                endtag = ''
                doc = ''

### 추출할 element

patent_id : 첫 doc-number  
app_cited : patcit / document-id / country='US' & len(doc-number) > 7  
* ** country != 'US'에서 탈출 (다음 문서로 넘너가면 됨) **

examiner: citation / category  

In [4]:
sample = extract_xml_strings('ipgb20100105.xml')
doc1=sample.next()
root = et.fromstring(doc1)
n = root.iter('doc-number')
pid = n.next().text
pid

'D7643386'

pid의 문제: 맨 앞이 D로 시작하는 애들 때문에 D없는 애들 앞에 0이 붙음  
but, 우리가 관심있는 특허는 D로 시작하는 애들 없음  
**=> int 만들어서 0 떼고 value Error 뜨면 바로 다음 doc으로 넘겨버리자**

In [4]:
sample = extract_xml_strings('ipgb20100105.xml')
a=[]
b=0
c=0
d=0
e=0
f=0
for doc in sample:
    root = et.fromstring(doc)
    b += 1
    n = root.iter('doc-number')
    c += 1
    pid = n.next().text
    d += 1
    try:
        pid = int(pid)
        e += 1
        a.append(pid)

    except ValueError:
        f += 1
        pass
        
a,b,c,d,e,f

([7643387], 2, 2, 2, 1, 1)

cit id의 문제: 연도와 번호 사이의 /  
**=> / 를 공백으로 만들자**

In [21]:
cit=root.iter('citation')
k = cit.next()
did = k[0][0]
cited = k[1].text
a=did.find('doc-number').text
a, cited

('5673247', 'cited by examiner')

In [22]:
cit_dum = 1 if cited.split()[-1] == 'examiner' else 0
cit_dum

1

In [8]:
a=a.replace('/','')
a

'5673247'

In [3]:
date = '20100105'  #initializing date

with open('app_citation.csv', 'wb') as cit:
    writer = csv.writer(cit,quoting=csv.QUOTE_ALL)
    writer.writerow(['patent_id','app_cited','by_examiner'])
    
    while not date=='20100126':
        filename = 'xmls/ipgb'+ date + '.xml'   #making filename
        xml = extract_xml_strings(filename)   #load and split whole xml into app_xmls  
        for doc in xml:
            root = et.fromstring(doc)
            n = root.iter('doc-number')
            pid = n.next().text

            try:
                pid = int(pid)                          
                for cit in root.iter('citation'):   #using iter because of multi-assignee apps 
                    did = cit[0][0]
                    try:
                        cited = cit[1].text              
                        try:
                            if did.find('country').text != 'US':  #if foreign cit, go to next doc
                                break 

                            cid = did.find('doc-number').text
                            if len(cid) > 8:     #if app cit, go to next doc
                                break
                            
                            cit_dum = 1 if cited.split()[-1] == 'examiner' else 0

                            writer.writerow([pid,cid,cit_dum])

                        except AttributeError:         #if other cit, go to next doc
                            break
                        
                    except IndexError:    #if other cit, go to next doc
                        break
                        
            except ValueError:      #if pid != int, go to the next doc
                continue    

        date = dt.datetime.strptime(date,'%Y%m%d')   #Let's move on to next xml
        date += dt.timedelta(weeks=1)
        date = date.strftime('%Y%m%d')   

In [7]:
date

'20100126'

# regex 써서 patent infringement case 개수 확인

In [1]:
import csv
import os
import re

os.chdir('J:/data')

patent_regex = re.compile('.* Patent Infringement$')
cnt = 0

In [2]:
with open('cases.csv', 'rb') as f:
    reader = csv.reader(f)
    header = reader.next()
    i = header.index('case_cause')
    for line in reader:
        if patent_regex.findall(line[i]):
            cnt+=1
cnt

6

In [3]:
i

7

# 기업간 proximity matrix 제작

In [1]:
import pandas as pd
import numpy as np
import csv
import os
import math
from math import sqrt

os.chdir('J:/data')

In [2]:
def prox_numer(l1,l2):
    cnt = 0.0
    for i in xrange(len(l1)):
        if l1[i]!=0 and l2[i]!=0:
            w= float(l1[i]) * l2[i]
            cnt += w
    
    return round(cnt,3) 

def prox_denom(l1,l2):
    a = sqrt(sum(map(lambda x: x**2, l1)))
    b = sqrt(sum(map(lambda x: x**2, l2)))
    
    return a*b 

def proximity(l1,l2):
    return prox_numer(l1,l2) / prox_denom(l1,l2)

def get_prox(l1,arr):
    sim = []
    for row in arr:
        sim.append(proximity(l1,row))
        
    return sim

In [9]:
arr = pd.read_csv('sample.csv').values

header = arr[:,0].tolist()
header = ['gvkey'] + map(int,header)

arr = np.delete(arr,0,1)

with open('counting.csv','wb') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    
    i=1  
    for row in arr:        
        line = get_prox(row,arr)
        line = [header[i]] + line             
        writer.writerow(line)
        i += 1